## 3 Consuming data using Kafka

Implement an Apache Kafka consumer to consume the data from section 2.8b. Create a program to consume the latest data and display the sensor locations on a map. 

### Import library

In [8]:
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import plotly.express as px
import numpy as np
import time
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import chart_studio.plotly as py
import folium
from IPython.display import display, HTML
from os import system, name
import os
from IPython.display import clear_output

### Comsume the streaming data with Kafka consumer

In [11]:
# Set topic name
topic1 = 'ped_count_2000'

# Create function for to connect the streaming application with the consumer
def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic1,
                                   consumer_timeout_ms=10000, # stop iteration if no message after 10 sec
                                   auto_offset_reset='earliest', # comment this if you don't want to consume earliest available message
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer


In [12]:
# Create funciton to read data in the consumer
def consume_messages(consumer):
    try:
        # Create map base for markers
        for message in consumer:   
            m1 = folium.Map(width=550,height=350,location=[-37.8136, 144.9631],zoom_start=15,min_zoom=8,max_zoom=20)
            # extract latitude, longitude and sensor id from the message day by day
            data = str(message.value.decode('utf-8')).split(', ')
            x, y, z, r = [], [], [], []

            for i in range(len(data)):
                u = data[i].split(',')
                lat = float(u[7].partition(':')[-1].split()[0])
                lon = float(u[8].partition(':')[-1].split()[0])
                sen_id = int(u[6].partition(':')[-1].split()[0])
                date1 = [u[0].partition(':')[-1].split()[0]]
                x.append(lat)
                y.append(lon)
                z.append(sen_id)
                r.append(date1)
           
             # Plot marker with the latitude and longitude
            for i in range(len(x)):
                folium.Marker(location=[x[i],y[i]],popup =['Sensor ID:', z[i], r[i]],tooltip='<strong>Click here to see Popup</strong>',icon=folium.Icon(color='red',icon='none')).add_to(m1)
            display(m1)
            
            # sleep for 5 seconds after printing output 
            sleep(3)
            
            # Clear outplot every 5 seconds
            clear_output(wait=True)

    except Exception as ex:
        print(str(ex))
       
    
if __name__ == '__main__':
    
    consumer = connect_kafka_consumer()
    consume_messages(consumer)
   